In [45]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [46]:
cwl_dir = '../cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [47]:
template = {
    'maf': {
            'class': 'File',
            'path': 'a/file/path'
    },
    'bam': {
            'class': 'File',
            'path': 'a/file/path'
    },
    'ref_dir': {
        'class': 'Directory',
        'path': 'a/dir/path'
    },
    'bed': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'input_type': 'a_string' # rna or dna, default is dna
}

In [48]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.neoscan.yaml'), 'w'))

In [57]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    'id': 'neoscan',
    'label': 'neoscan',
    'baseCommand': ['/usr/bin/python', '/pecgs-neoscan/src/neoscan.py'],
    'inputs': [
        {
            'id': 'maf',
            'type': 'File',
            'inputBinding': {
                'position': '1',
            }
        },
        {
            'id': 'bam',
            'type': 'File',
            'secondaryFiles': [
               '$(self.basename).bai',
            ],
            'inputBinding': {
                'position': '2',
            }
        },
        {
            'id': 'ref_dir',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--ref-dir',
                'position': '0'
            }
        },
        {
            'id': 'bed',
            'type': 'File',
            'inputBinding': {
                'prefix': '--bed',
                'position': '0'
            }
        },
        {
            'id': 'input_type',
            'type': 'string?',
            'default': 'dna',
            'inputBinding': {
                'prefix': '--input-type',
                'position': '0'
            }
        },
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
#             'default': '/miniconda/envs/neoscan/bin:$PATH'
            'default': '/usr/local/bin/OptiType:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/home/biodocker/bin:/miniconda/envs/neoscan/bin:/miniconda/bin:$PATH' # just hardcoding here is the easiest thing for now
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--out-dir',
            'valueFrom': 'output'
        },
        {
            'position': 0,
            'prefix': '--log-dir',
            'valueFrom': 'logs'
        },
        {
            'position': 0,
            'prefix': '--neoscan-dir',
            'valueFrom': '/pecgs-neoscan/src/neoscan'
        },
        {
            'position': 0,
            'prefix': '--optitype-script',
            'valueFrom': '/usr/local/bin/OptiType/OptiTypePipeline.py'
        },
        {
            'position': 0,
            'prefix': '--f-opti-config',
            'valueFrom': '/pecgs-neoscan/src/neoscan/config.ini'
        },
        {
            'position': 0,
            'prefix': '--netmhc',
            'valueFrom': '/pecgs-neoscan/src/netmhc/netMHC-4.0/netMHC'
        },
        {
            'position': 0,
            'prefix': '--f-allele',
            'valueFrom': '/pecgs-neoscan/src/netmhc/netMHC-4.0/data/allelelist'
        },
        
    ],
    'outputs': [
        {
            'id': 'snv_summary',
            'type': 'File',
            'outputBinding': {'glob': 'output/sample/sample.neo.snv.summary'},
        },
        {
            'id': 'indel_summary',
            'type': 'File',
            'outputBinding': {'glob': 'output/sample/sample.neo.indel.summary'},
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs-neoscan:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 100000,
            'coresMin': 4
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [58]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'neoscan.cwl'), 'w'))